In [1]:
!pip install -U numpy==1.23.5

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install -q git+https://github.com/sunsmarterjie/yolov12.git roboflow supervision flash-attn

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 85.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip uninstall -y numpy

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2


In [4]:
!nvidia-smi

import os
HOME = os.getcwd()
print(HOME)

# !pip install ultralytics==8.0.20

from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from IPython.display import display, Image

# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

# SOURCE_VIDEO_PATH = f"{HOME}/vehicle-counting.mp4"

#SOURCE_VIDEO_PATH = f"/content/drive/MyDrive/VIDEOS FLORICOLA /GX010697.MP4"
!pip install loguru

%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git

###
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
# !sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

# !pip3 install -q -r requirements.txt
# !python3 setup.py -q develop
# !pip install -q cython_bbox
# !pip install -q onemetric
###

# !cd ByteTrack && pip3 install -q -r requirements.txt
# !cd ByteTrack && python3  setup.py  -q develop
# !pip install -q cython_bbox
!pip install -q onemetric

!pip3 install cython
!pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip3 install cython_bbox

%cd /content/ByteTrack/
!pip3 install -r requirements.txt

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

!pip install lap

from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    # track_thresh: float = 0.1
    # track_buffer: int =80
    # match_thresh: float = 0.8
    # aspect_ratio_thresh: float = 2.0
    # min_box_area: float = 5.0
    # mot20: bool = False


#valores originales
    track_thresh: float = 0.2
    track_buffer: int = 30
    match_thresh: float = 0.95
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 2.0
    mot20: bool = False


!pip install supervision==0.1.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

from typing import List

import numpy as np

supervision.__version__: 0.1.0


In [ ]:
SOURCE_VIDEO_PATH = f"/content/drive/MyDrive/TRABAJO UNACEM 2025/PROYECTOS-ENVASE/2025/Videos/11-06-25"


In [5]:
# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)
    tracker_id_dict = {}
    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids


MODEL="/content/drive/MyDrive/TRABAJO UNACEM 2025/PROYECTOS-ENVASE/2025/modelos/YOLO8/ACS-V3.pt"

from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

CLASS_NAMES_DICT = model.model.names
# change label for class_id 0 to "rosa"
# CLASS_NAMES_DICT[0] = "sepalo_desprendido"
# CLASS_NAMES_DICT[1] = "garbanzo"
# CLASS_NAMES_DICT[2] = "punto_uno"
# CLASS_NAMES_DICT[3] = "raya_color"
#CLASS_NAMES_DICT[4] = "sépalo_desprendido"

CLASS_NAMES_DICT[0] = "Armaduro"
CLASS_NAMES_DICT[1] = "Selvalegre"
CLASS_NAMES_DICT[2] = "Campeon"

CLASS_ID = [0,1,2]

Model summary (fused): 268 layers, 68,126,457 parameters, 0 gradients, 257.4 GFLOPs


In [6]:
# create frame generator
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
# create instance of BoxAnnotator
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=1)
# acquire first video frame
iterator = iter(generator)
frame = next(iterator)
# model prediction on single frame and conversion to supervision Detections
results = model(frame)
detections = Detections(
    xyxy=results[0].boxes.xyxy.cpu().numpy(),
    confidence=results[0].boxes.conf.cpu().numpy(),
    class_id=results[0].boxes.cls.cpu().numpy().astype(int)
)
# format custom labels
labels = [
    f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for _, confidence, class_id, tracker_id
    in detections
]


# annotate and display frame
frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)

%matplotlib inline
show_frame_in_notebook(frame, (10, 10))


NameError: name 'SOURCE_VIDEO_PATH' is not defined

In [ ]:
ruta_carpeta_salida = "/content/drive/MyDrive/TRABAJO UNACEM 2025/PROYECTOS-ENVASE/2025/Resultados"

# Guardamos la imagen con las predicciones
nombre_nuevo_archivo = "Resultado-7-"
nombre_nuevo_archivo2 = f'{nombre_nuevo_archivo}.mp4'
TARGET_VIDEO_PATH = os.path.join(ruta_carpeta_salida, nombre_nuevo_archivo2)

**Un Solo Video**

In [ ]:
ruta_carpeta_salida = "/content/drive/MyDrive/TRABAJO UNACEM 2025/PROYECTOS-ENVASE/2025/Resultados"
# Guardamos la imagen con las predicciones
nombre_nuevo_archivo = "Resultado-7-"
nombre_nuevo_archivo2 = f'{nombre_nuevo_archivo}.mp4'


TARGET_VIDEO_PATH = os.path.join(ruta_carpeta_salida, nombre_nuevo_archivo2)

# ACTUALIZACIÓN LIBRERÍA

import cv2
from typing import List
import numpy as np
import pandas as pd
from openpyxl import Workbook
from openpyxl.chart import BarChart, Reference
from openpyxl.utils.dataframe import dataframe_to_rows

result_df = pd.DataFrame()

# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))

def detections2boxes(detections: Detections) -> np.ndarray:
    confidence_with_new_axis = detections.confidence.reshape(-1, 1)  # Crear un nuevo eje usando reshape
    return np.hstack((detections.xyxy, confidence_with_new_axis))

# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track in tracks
    ], dtype=float)

# matches our bounding boxes with predictions
def match_detections_with_tracks(detections: Detections, tracks: List[STrack]) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            class_id = detections.class_id[detection_index]
            if class_id in [0, 1, 2,3,4]:
                tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

from tqdm.notebook import tqdm

# create BYTETracker instance
byte_tracker = BYTETracker(BYTETrackerArgs())
# create VideoInfo instance
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
# create frame generator
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)

# create instance of BoxAnnotator and LineCounterAnnotator
box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)
line_annotator = LineCounterAnnotator(thickness=4, text_thickness=4, text_scale=2)

w,h=frame.shape[1],frame.shape[0]
# print("w",w)
# print("h",h)

linea_conteo=round(w/2,None)

start_point = (linea_conteo, 0)
end_point = (linea_conteo, h)

# start_point = (w, 0)
# end_point = (0, h)

start_point = (w // 2, 0)  # Comienza desde el centro horizontal y en la parte superior
end_point = (w // 2, h)    # Termina en el centro horizontal y en la parte inferior

contador_selvalegre=0
contador_campeon=0

data=[]

# Crear un nuevo archivo de Excel y obtener la hoja de trabajo activa
wb = Workbook()
ws = wb.active
ws.title = 'Contadores'
# Create an empty DataFrame
#result_df = pd.DataFrame(columns=['Frame', 'Contador_Garbanzos', 'Contador_Punto_Uno', 'Contador_Cepalo_Desprendido', 'Contador_Rosa_Marchita', 'Contador_Raya_Color'])

# open target video file
with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    # loop over video frames
    for frame in tqdm(generator, total=video_info.total_frames):
        # model prediction on single frame and conversion to supervision Detections
        results = model(frame)
        detections = Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int)
        )
        # filtering out detections with unwanted classes
        mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        # tracking detections
        tracks = byte_tracker.update(
            output_results=detections2boxes(detections=detections),
            img_info=frame.shape,
            img_size=frame.shape
        )

        tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
        detections.tracker_id = np.array(tracker_id)
        # filtering out detections without trackers
        mask = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
        detections.filter(mask=mask, inplace=True)
        detections.filter(mask=np.array([t is not None for t in detections.tracker_id], dtype=bool), inplace=True)
        # format custom labels
        labels = [
            f" {CLASS_NAMES_DICT[class_id]}"
            for _, confidence, class_id, tracker_id
            in detections
        ]
        # Agrega una línea entre los dos puntos en el marco
        frame_with_line = cv2.line(frame, start_point, end_point, color=(255, 255, 255), thickness=2)
        # annotate and display frame

        for class_id, xyxy, tracker_id in zip(detections.class_id, detections.xyxy, detections.tracker_id):
          #print("class_id", CLASS_NAMES_DICT[class_id])
          if CLASS_NAMES_DICT[class_id] == "Selvalegre":
              x_center = int(xyxy[0] + (xyxy[2] - xyxy[0]) / 2)
              if x_center >= linea_conteo:
              #if x_center >= linea_conteo or x_center < linea_conteo :
                  # El objeto cruzó la línea de izquierda a derecha
                  if tracker_id not in data:
                      contador_selvalegre += 1
                      data.append(tracker_id)
                      #print("contador garbanzos", contador_garbanzos)

          elif CLASS_NAMES_DICT[class_id] == "Campeon":
              x_center = int(xyxy[0] + (xyxy[2] - xyxy[0]) / 2)
              if x_center >= linea_conteo:
                if tracker_id not in data:
                    contador_campeon += 1
                    data.append(tracker_id)
                    #print("contador punto", contador_punto_uno)


        ancho_letra_blanca=2
        ancho_letra_contorno=4
        letra=0.9

        cv2.putText(frame, f'Selvalegre:'+str(contador_selvalegre), (20,30),cv2.FONT_HERSHEY_SIMPLEX, letra, (0,0,0), ancho_letra_contorno, cv2.LINE_AA)
        cv2.putText(frame, f'Selvalegre:'+str(contador_selvalegre), (20,30), cv2.FONT_HERSHEY_SIMPLEX,letra, (255,255,255), ancho_letra_blanca)

        cv2.putText(frame, f'Campeon:'+str(contador_campeon), (20,60),cv2.FONT_HERSHEY_SIMPLEX, letra, (0,0,0), ancho_letra_contorno, cv2.LINE_AA)
        cv2.putText(frame, f'Campeon:'+str(contador_campeon), (20,60), cv2.FONT_HERSHEY_SIMPLEX,letra, (255,255,255), ancho_letra_blanca)

        frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
        box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=1)
        # line_annotator.annotate(frame=frame, line_counter=line_counter)
        # guarda los frames procesados del video
        #sink.write_frame(frame_with_line)
        sink.write_frame(frame)

# Update the DataFrame with a new row for each variable
result_df = pd.concat([result_df, pd.DataFrame({'Tipo': 'Selvalegre', 'Conteo Total': [contador_selvalegre]})], ignore_index=True)
result_df = pd.concat([result_df, pd.DataFrame({'Tipo': 'Campeón', 'Conteo Total': [contador_campeon]})], ignore_index=True)

# Escribir el DataFrame en la hoja de trabajo de Excel
for row in dataframe_to_rows(result_df, index=False, header=True):
  ws.append(row)

# Crear un gráfico de barras
chart = BarChart()
chart.title = "Conteos Totales"
chart.x_axis.title = 'Tipo'
chart.y_axis.title = 'Contadores'

# Definir las referencias para los datos del gráfico
data = Reference(ws, min_col=2, min_row=1, max_col=2, max_row=result_df.shape[0]+1)
labels = Reference(ws, min_col=1, min_row=2, max_row=result_df.shape[0]+1)

# Agregar datos y etiquetas al gráfico
chart.add_data(data, titles_from_data=True)
chart.set_categories(labels)

# Añadir el gráfico a la hoja de trabajo de Excel
ws.add_chart(chart, "D1")

# Specify the output folder
#output_folder = '/content/drive/MyDrive/VIDEOS FLORICOLA /videos procesados'
# Guardar el archivo de Excel
wb.save(f'{ruta_carpeta_salida }/Conteo-total-{nombre_nuevo_archivo}.xlsx')
print("Archivo de excel exportado correctamente")

NameError: name 'SOURCE_VIDEO_PATH' is not defined

**Para Carpetas**

In [ ]:
# BIEN
import os
import cv2
import numpy as np
import pandas as pd
from openpyxl import Workbook
from openpyxl.chart import BarChart, Reference
from openpyxl.utils.dataframe import dataframe_to_rows
from typing import List
from tqdm.notebook import tqdm

# Carpeta de entrada y salida
ruta_carpeta_videos = "/content/drive/MyDrive/TRABAJO UNACEM 2025/PROYECTOS-ENVASE/2025/Videos/Arnaduro/18-06"
ruta_carpeta_salida = "/content/drive/MyDrive/TRABAJO UNACEM 2025/PROYECTOS-ENVASE/2025/Resultados/Resultado-18-06"

# Recorremos todos los archivos de video
for archivo in os.listdir(ruta_carpeta_videos):
    if archivo.endswith(".mp4"):  # o .avi, etc.
        nombre_base = os.path.splitext(archivo)[0]
        SOURCE_VIDEO_PATH = os.path.join(ruta_carpeta_videos, archivo)
        TARGET_VIDEO_PATH = os.path.join(ruta_carpeta_salida, f"{nombre_base}_procesado5.mp4")
        nombre_excel = os.path.join(ruta_carpeta_salida, f"Conteo-total5-{nombre_base}.xlsx")

        # Inicialización de contadores
        contador_selvalegre = 0
        contador_campeon = 0
        contador_armaduro= 0
        data = []
        result_df = pd.DataFrame()

        # Instancias de tracking y anotación
        byte_tracker = BYTETracker(BYTETrackerArgs())
        video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
        generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
        box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)

        # Obtener primer frame para calcular dimensiones y línea
        frame = next(generator)
        w, h = frame.shape[1], frame.shape[0]
        # linea_conteo = w // 2
        # start_point = (linea_conteo, 0)
        # end_point = (linea_conteo, h)


        linea_conteo = h // 2
        start_point = (0, linea_conteo)
        end_point = (w, linea_conteo)


        # Reiniciar generador porque ya consumimos un frame
        generator = get_video_frames_generator(SOURCE_VIDEO_PATH)

        # Excel Workbook
        wb = Workbook()
        ws = wb.active
        ws.title = 'Contadores'

        # Procesamiento del video
        with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
            for frame in tqdm(generator, total=video_info.total_frames):
                results = model(frame)
                detections = Detections(
                    xyxy=results[0].boxes.xyxy.cpu().numpy(),
                    confidence=results[0].boxes.conf.cpu().numpy(),
                    class_id=results[0].boxes.cls.cpu().numpy().astype(int)
                )

                # Filtrado
                mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
                detections.filter(mask=mask, inplace=True)

                # Tracking
                tracks = byte_tracker.update(
                    output_results=detections2boxes(detections),
                    img_info=frame.shape,
                    img_size=frame.shape
                )


                tracker_id = match_detections_with_tracks(detections, tracks)
                detections.tracker_id = np.array(tracker_id)

                # Asegurar que sea tipo booleano
                mask = np.array([t is not None for t in detections.tracker_id], dtype=bool)
                detections.filter(mask=mask, inplace=True)



                # tracker_id = match_detections_with_tracks(detections, tracks)
                # detections.tracker_id = np.array(tracker_id)
                # detections.filter(mask=np.array([t is not None for t in detections.tracker_id]), inplace=True)

                # Etiquetas
                labels = [f" {CLASS_NAMES_DICT[class_id]}" for _, conf, class_id, tracker_id in detections]

                # Línea de conteo
                frame_with_line = cv2.line(frame, start_point, end_point, (255, 255, 255), 2)

                # Conteo personalizado
                for class_id, xyxy, tracker_id in zip(detections.class_id, detections.xyxy, detections.tracker_id):
                    x_center = int(xyxy[0] + (xyxy[2] - xyxy[0]) / 2)
                    if x_center >= linea_conteo:
                        if tracker_id not in data:
                            data.append(tracker_id)
                            if CLASS_NAMES_DICT[class_id] == "Selvalegre":
                                contador_selvalegre += 1
                            elif CLASS_NAMES_DICT[class_id] == "Campeon":
                                contador_campeon += 1
                            elif CLASS_NAMES_DICT[class_id] == "Armaduro":
                                contador_armaduro += 1


                ancho_letra_blanca=2
                ancho_letra_contorno=4
                letra=0.9

                cv2.putText(frame, f'Selvalegre:'+str(contador_selvalegre), (20,30),cv2.FONT_HERSHEY_SIMPLEX, letra, (0,0,0), ancho_letra_contorno, cv2.LINE_AA)
                cv2.putText(frame, f'Selvalegre:'+str(contador_selvalegre), (20,30), cv2.FONT_HERSHEY_SIMPLEX,letra, (255,255,255), ancho_letra_blanca)

                cv2.putText(frame, f'Campeon:'+str(contador_campeon), (20,60),cv2.FONT_HERSHEY_SIMPLEX, letra, (0,0,0), ancho_letra_contorno, cv2.LINE_AA)
                cv2.putText(frame, f'Campeon:'+str(contador_campeon), (20,60), cv2.FONT_HERSHEY_SIMPLEX,letra, (255,255,255), ancho_letra_blanca)

                cv2.putText(frame, f'Armaduro:'+str(contador_armaduro), (20,90),cv2.FONT_HERSHEY_SIMPLEX, letra, (0,0,0), ancho_letra_contorno, cv2.LINE_AA)
                cv2.putText(frame, f'Armaduro:'+str(contador_armaduro), (20,90), cv2.FONT_HERSHEY_SIMPLEX,letra, (255,255,255), ancho_letra_blanca)

                frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
                box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=1)
                sink.write_frame(frame)

        # Guardar en Excel
        result_df = pd.concat([result_df, pd.DataFrame({'Tipo': 'Selvalegre', 'Conteo Total': [contador_selvalegre]})], ignore_index=True)
        result_df = pd.concat([result_df, pd.DataFrame({'Tipo': 'Campeón', 'Conteo Total': [contador_campeon]})], ignore_index=True)
        result_df = pd.concat([result_df, pd.DataFrame({'Tipo': 'Armaduro', 'Conteo Total': [contador_armaduro]})], ignore_index=True)

        for row in dataframe_to_rows(result_df, index=False, header=True):
            ws.append(row)

        chart = BarChart()
        chart.title = "Conteos Totales"
        chart.x_axis.title = 'Tipo'
        chart.y_axis.title = 'Contadores'
        data_ref = Reference(ws, min_col=2, min_row=1, max_row=result_df.shape[0]+1)
        labels = Reference(ws, min_col=1, min_row=2, max_row=result_df.shape[0]+1)
        chart.add_data(data_ref, titles_from_data=True)
        chart.set_categories(labels)
        ws.add_chart(chart, "D1")

        wb.save(nombre_excel)
        print(f"✅ Procesado y exportado: {archivo}")


In [9]:
import os
import cv2
import numpy as np
import pandas as pd
from openpyxl import Workbook
from openpyxl.chart import BarChart, Reference
from openpyxl.utils.dataframe import dataframe_to_rows
from typing import List
from tqdm.notebook import tqdm

# Carpeta de entrada y salida
ruta_carpeta_videos = "/content/drive/MyDrive/TRABAJO UNACEM 2025/PROYECTOS-ENVASE/2025/Videos/Arnaduro/18-06"
ruta_carpeta_salida = "/content/drive/MyDrive/TRABAJO UNACEM 2025/PROYECTOS-ENVASE/2025/Resultados/Resultado-18-06"

# Recorremos todos los archivos de video
for archivo in os.listdir(ruta_carpeta_videos):
    if archivo.endswith(".mp4"):
        nombre_base = os.path.splitext(archivo)[0]
        SOURCE_VIDEO_PATH = os.path.join(ruta_carpeta_videos, archivo)
        TARGET_VIDEO_PATH = os.path.join(ruta_carpeta_salida, f"{nombre_base}_procesado2.mp4")
        nombre_excel = os.path.join(ruta_carpeta_salida, f"Conteo-total2-{nombre_base}.xlsx")

        contador_selvalegre = 0
        contador_campeon = 0
        contador_armaduro = 0
        data = []
        posiciones_previas = {}
        result_df = pd.DataFrame()

        byte_tracker = BYTETracker(BYTETrackerArgs())
        video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
        generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
        box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)

        frame = next(generator)
        w, h = frame.shape[1], frame.shape[0]
        linea_conteo = h // 2
        start_point = (0, linea_conteo)
        end_point = (w, linea_conteo)

        generator = get_video_frames_generator(SOURCE_VIDEO_PATH)

        wb = Workbook()
        ws = wb.active
        ws.title = 'Contadores'

        with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
            for frame in tqdm(generator, total=video_info.total_frames):
                results = model(frame)
                detections = Detections(
                    xyxy=results[0].boxes.xyxy.cpu().numpy(),
                    confidence=results[0].boxes.conf.cpu().numpy(),
                    class_id=results[0].boxes.cls.cpu().numpy().astype(int)
                )

                mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
                detections.filter(mask=mask, inplace=True)

                tracks = byte_tracker.update(
                    output_results=detections2boxes(detections),
                    img_info=frame.shape,
                    img_size=frame.shape
                )

                tracker_id = match_detections_with_tracks(detections, tracks)
                detections.tracker_id = np.array(tracker_id)
                mask = np.array([t is not None for t in detections.tracker_id], dtype=bool)
                detections.filter(mask=mask, inplace=True)

                labels = [f" {CLASS_NAMES_DICT[class_id]}" for _, conf, class_id, tracker_id in detections]
                frame_with_line = cv2.line(frame, start_point, end_point, (255, 255, 255), 2)

                for class_id, xyxy, tracker_id in zip(detections.class_id, detections.xyxy, detections.tracker_id):
                    x_center = int(xyxy[0] + (xyxy[2] - xyxy[0]) / 2)
                    y_center = int(xyxy[1] + (xyxy[3] - xyxy[1]) / 2)

                    y_anterior = posiciones_previas.get(tracker_id, None)

                    if y_anterior is not None:
                        if y_anterior < linea_conteo and y_center >= linea_conteo:
                            if tracker_id not in data:
                                data.append(tracker_id)
                                if CLASS_NAMES_DICT[class_id] == "Selvalegre":
                                    contador_selvalegre += 1
                                elif CLASS_NAMES_DICT[class_id] == "Campeon":
                                    contador_campeon += 1
                                elif CLASS_NAMES_DICT[class_id] == "Armaduro":
                                    contador_armaduro += 1

                    posiciones_previas[tracker_id] = y_center

                ancho_letra_blanca = 1
                ancho_letra_contorno = 2
                letra = 0.6

                cv2.putText(frame, f'Selvalegre:' + str(contador_selvalegre), (18, 55), cv2.FONT_HERSHEY_SIMPLEX, letra, (0, 0, 0), ancho_letra_contorno, cv2.LINE_AA)
                cv2.putText(frame, f'Selvalegre:' + str(contador_selvalegre), (18, 55), cv2.FONT_HERSHEY_SIMPLEX, letra, (255, 255, 255), ancho_letra_blanca)

                cv2.putText(frame, f'Campeon:' + str(contador_campeon), (18, 80), cv2.FONT_HERSHEY_SIMPLEX, letra, (0, 0, 0), ancho_letra_contorno, cv2.LINE_AA)
                cv2.putText(frame, f'Campeon:' + str(contador_campeon), (18, 80), cv2.FONT_HERSHEY_SIMPLEX, letra, (255, 255, 255), ancho_letra_blanca)

                cv2.putText(frame, f'Armaduro:' + str(contador_armaduro), (18, 105), cv2.FONT_HERSHEY_SIMPLEX, letra, (0, 0, 0), ancho_letra_contorno, cv2.LINE_AA)
                cv2.putText(frame, f'Armaduro:' + str(contador_armaduro), (18, 105), cv2.FONT_HERSHEY_SIMPLEX, letra, (255, 255, 255), ancho_letra_blanca)

                frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
                box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=1)
                sink.write_frame(frame)

        result_df = pd.concat([result_df, pd.DataFrame({'Tipo': 'Selvalegre', 'Conteo Total': [contador_selvalegre]})], ignore_index=True)
        result_df = pd.concat([result_df, pd.DataFrame({'Tipo': 'Campeón', 'Conteo Total': [contador_campeon]})], ignore_index=True)
        result_df = pd.concat([result_df, pd.DataFrame({'Tipo': 'Armaduro', 'Conteo Total': [contador_armaduro]})], ignore_index=True)

        for row in dataframe_to_rows(result_df, index=False, header=True):
            ws.append(row)

        chart = BarChart()
        chart.title = "Conteos Totales"
        chart.x_axis.title = 'Tipo'
        chart.y_axis.title = 'Contadores'
        data_ref = Reference(ws, min_col=2, min_row=1, max_row=result_df.shape[0] + 1)
        labels = Reference(ws, min_col=1, min_row=2, max_row=result_df.shape[0] + 1)
        chart.add_data(data_ref, titles_from_data=True)
        chart.set_categories(labels)
        ws.add_chart(chart, "D1")

        wb.save(nombre_excel)
        print(f"✅ Procesado y exportado: {archivo}")


  0%|          | 0/1619 [00:00<?, ?it/s]


0: 384x640 1 Armaduro, 63.0ms
Speed: 1.9ms preprocess, 63.0ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Armaduro, 63.7ms
Speed: 1.7ms preprocess, 63.7ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Armaduro, 43.1ms
Speed: 1.3ms preprocess, 43.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Armaduro, 43.1ms
Speed: 1.0ms preprocess, 43.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Armaduro, 43.1ms
Speed: 1.1ms preprocess, 43.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Armaduro, 40.8ms
Speed: 1.5ms preprocess, 40.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Armaduros, 35.1ms
Speed: 1.3ms preprocess, 35.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Armaduros, 34.6ms
Speed: 1.1ms preprocess, 34.6ms inference, 1.3ms postprocess per image 

**Detecciones con OCR**

In [ ]:
!pip install pytesseract



In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import pytesseract  # Importamos la librería OCR
from openpyxl import Workbook
from openpyxl.chart import BarChart, Reference
from openpyxl.utils.dataframe import dataframe_to_rows
from typing import List
from tqdm.notebook import tqdm

# Carpeta de entrada y salida
ruta_carpeta_videos = "/content/drive/MyDrive/TRABAJO UNACEM 2025/PROYECTOS-ENVASE/2025/Videos/Muestra-07-04-2025"
ruta_carpeta_salida = "/content/drive/MyDrive/TRABAJO UNACEM 2025/PROYECTOS-ENVASE/2025/Resultados-videos-4"

# Recorremos todos los archivos de video
for archivo in os.listdir(ruta_carpeta_videos):
    if archivo.endswith(".mp4"):  # o .avi, etc.
        nombre_base = os.path.splitext(archivo)[0]
        SOURCE_VIDEO_PATH = os.path.join(ruta_carpeta_videos, archivo)
        TARGET_VIDEO_PATH = os.path.join(ruta_carpeta_salida, f"{nombre_base}_procesado.mp4")
        nombre_excel = os.path.join(ruta_carpeta_salida, f"Conteo-total-{nombre_base}.xlsx")

        # Inicialización de contadores
        contador_selvalegre = 0
        contador_campeon = 0
        data = []
        result_df = pd.DataFrame()

        # Instancias de tracking y anotación
        byte_tracker = BYTETracker(BYTETrackerArgs())
        video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
        generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
        box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)

        # Obtener primer frame para calcular dimensiones y línea
        frame = next(generator)
        w, h = frame.shape[1], frame.shape[0]
        # linea_conteo = w // 2
        # start_point = (linea_conteo, 0)
        # end_point = (linea_conteo, h)


        linea_conteo = h // 2
        start_point = (0, linea_conteo)
        end_point = (w, linea_conteo)

        # Reiniciar generador porque ya consumimos un frame
        generator = get_video_frames_generator(SOURCE_VIDEO_PATH)

        # Excel Workbook
        wb = Workbook()
        ws = wb.active
        ws.title = 'Contadores'

        # Procesamiento del video
        with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
            for frame in tqdm(generator, total=video_info.total_frames):
                results = model(frame)
                detections = Detections(
                    xyxy=results[0].boxes.xyxy.cpu().numpy(),
                    confidence=results[0].boxes.conf.cpu().numpy(),
                    class_id=results[0].boxes.cls.cpu().numpy().astype(int)
                )

                # Filtrado
                mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
                detections.filter(mask=mask, inplace=True)

                # Tracking
                tracks = byte_tracker.update(
                    output_results=detections2boxes(detections),
                    img_info=frame.shape,
                    img_size=frame.shape
                )

                tracker_id = match_detections_with_tracks(detections, tracks)
                detections.tracker_id = np.array(tracker_id)

                # Asegurar que sea tipo booleano
                mask = np.array([t is not None for t in detections.tracker_id], dtype=bool)
                detections.filter(mask=mask, inplace=True)

                # Etiquetas
                labels = [f" {CLASS_NAMES_DICT[class_id]}" for _, conf, class_id, tracker_id in detections]

                # Línea de conteo
                frame_with_line = cv2.line(frame, start_point, end_point, (255, 255, 255), 2)

                # Conteo personalizado
                for class_id, xyxy, tracker_id in zip(detections.class_id, detections.xyxy, detections.tracker_id):
                    x_center = int(xyxy[0] + (xyxy[2] - xyxy[0]) / 2)
                    if x_center >= linea_conteo:
                        if tracker_id not in data:
                            data.append(tracker_id)
                            # Extraemos la región de la imagen para OCR
                            crop_img = frame[int(xyxy[1]):int(xyxy[3]), int(xyxy[0]):int(xyxy[2])]

                            # Aplicamos OCR para leer el texto
                            ocr_text = pytesseract.image_to_string(crop_img, config='--psm 6').strip()

                            if "Selvalegre" in ocr_text:
                                contador_selvalegre += 1
                            elif "Campeon" in ocr_text:
                                contador_campeon += 1

                # Actualizar los contadores en el frame
                ancho_letra_blanca=2
                ancho_letra_contorno=4
                letra=0.9

                cv2.putText(frame, f'Selvalegre:'+str(contador_selvalegre), (20,30),cv2.FONT_HERSHEY_SIMPLEX, letra, (0,0,0), ancho_letra_contorno, cv2.LINE_AA)
                cv2.putText(frame, f'Selvalegre:'+str(contador_selvalegre), (20,30), cv2.FONT_HERSHEY_SIMPLEX,letra, (255,255,255), ancho_letra_blanca)

                cv2.putText(frame, f'Campeon:'+str(contador_campeon), (20,60),cv2.FONT_HERSHEY_SIMPLEX, letra, (0,0,0), ancho_letra_contorno, cv2.LINE_AA)
                cv2.putText(frame, f'Campeon:'+str(contador_campeon), (20,60), cv2.FONT_HERSHEY_SIMPLEX,letra, (255,255,255), ancho_letra_blanca)

                frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
                box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=1)
                sink.write_frame(frame)

        # Guardar en Excel
        result_df = pd.concat([result_df, pd.DataFrame({'Tipo': 'Selvalegre', 'Conteo Total': [contador_selvalegre]})], ignore_index=True)
        result_df = pd.concat([result_df, pd.DataFrame({'Tipo': 'Campeón', 'Conteo Total': [contador_campeon]})], ignore_index=True)

        for row in dataframe_to_rows(result_df, index=False, header=True):
            ws.append(row)

        chart = BarChart()
        chart.title = "Conteos Totales"
        chart.x_axis.title = 'Tipo'
        chart.y_axis.title = 'Contadores'
        data_ref = Reference(ws, min_col=2, min_row=1, max_row=result_df.shape[0]+1)
        labels = Reference(ws, min_col=1, min_row=2, max_row=result_df.shape[0]+1)
        chart.add_data(data_ref, titles_from_data=True)
        chart.set_categories(labels)
        ws.add_chart(chart, "D1")

        wb.save(nombre_excel)
        print(f"✅ Procesado y exportado: {archivo}")


  0%|          | 0/1177 [00:00<?, ?it/s]


0: 384x640 3 Selvalegres, 56.3ms
Speed: 15.0ms preprocess, 56.3ms inference, 316.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Selvalegres, 31.0ms
Speed: 2.4ms preprocess, 31.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Selvalegres, 21.7ms
Speed: 3.9ms preprocess, 21.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Selvalegres, 21.6ms
Speed: 2.2ms preprocess, 21.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Selvalegres, 23.2ms
Speed: 1.8ms preprocess, 23.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Selvalegres, 22.2ms
Speed: 1.9ms preprocess, 22.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Selvalegres, 24.9ms
Speed: 1.8ms preprocess, 24.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Selvalegres, 24.8ms
Speed: 1.9ms preprocess, 24.8ms inference, 1.6

  0%|          | 0/880 [00:00<?, ?it/s]


0: 384x640 2 Selvalegres, 17.0ms
Speed: 2.4ms preprocess, 17.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Selvalegres, 23.1ms
Speed: 2.4ms preprocess, 23.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Selvalegres, 18.7ms
Speed: 1.9ms preprocess, 18.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Selvalegres, 17.2ms
Speed: 3.4ms preprocess, 17.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Selvalegres, 17.0ms
Speed: 1.9ms preprocess, 17.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Selvalegres, 19.0ms
Speed: 1.9ms preprocess, 19.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Selvalegres, 18.1ms
Speed: 1.8ms preprocess, 18.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Selvalegres, 20.4ms
Speed: 2.2ms preprocess, 20.4ms inference, 1.2ms 

  0%|          | 0/1354 [00:00<?, ?it/s]


0: 384x640 (no detections), 37.5ms
Speed: 2.2ms preprocess, 37.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 38.4ms
Speed: 2.2ms preprocess, 38.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Selvalegre, 28.2ms
Speed: 3.7ms preprocess, 28.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Selvalegre, 26.2ms
Speed: 1.9ms preprocess, 26.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Selvalegre, 25.5ms
Speed: 3.2ms preprocess, 25.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Selvalegre, 23.1ms
Speed: 2.0ms preprocess, 23.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Selvalegre, 26.1ms
Speed: 1.9ms preprocess, 26.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Selvalegre, 34.7ms
Speed: 2.0ms preprocess, 34.7ms inference, 1.9ms po

  0%|          | 0/1300 [00:00<?, ?it/s]


0: 480x640 2 Selvalegres, 76.0ms
Speed: 4.2ms preprocess, 76.0ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 Selvalegres, 22.8ms
Speed: 3.8ms preprocess, 22.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 Selvalegres, 19.4ms
Speed: 7.6ms preprocess, 19.4ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 Selvalegres, 22.1ms
Speed: 3.9ms preprocess, 22.1ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 Selvalegres, 23.6ms
Speed: 3.8ms preprocess, 23.6ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 Selvalegres, 19.1ms
Speed: 5.6ms preprocess, 19.1ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 Selvalegres, 19.6ms
Speed: 4.6ms preprocess, 19.6ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 4 Selvalegres, 23.3ms
Speed: 8.5ms preprocess, 23.3ms inference, 1.3ms 

  0%|          | 0/2009 [00:00<?, ?it/s]

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Speed: 1.8ms preprocess, 22.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Campeon, 3 Selvalegres, 25.1ms
Speed: 1.8ms preprocess, 25.1ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Campeon, 3 Selvalegres, 25.5ms
Speed: 2.1ms preprocess, 25.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Campeon, 3 Selvalegres, 23.5ms
Speed: 1.8ms preprocess, 23.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Campeon, 3 Selvalegres, 19.8ms
Speed: 2.5ms preprocess, 19.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Selvalegres, 23.9ms
Speed: 2.1ms preprocess, 23.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Selvalegres, 24.7ms
Speed: 1.8ms preprocess, 24.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 38

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import pytesseract  # Importamos la librería OCR
from openpyxl import Workbook
from openpyxl.chart import BarChart, Reference
from openpyxl.utils.dataframe import dataframe_to_rows
from tqdm.notebook import tqdm

# Carpeta de entrada y salida
ruta_carpeta_videos = "/content/drive/MyDrive/TRABAJO UNACEM 2025/PROYECTOS-ENVASE/2025/Videos/Muestra-07-04-2025"
ruta_carpeta_salida = "/content/drive/MyDrive/TRABAJO UNACEM 2025/PROYECTOS-ENVASE/Resultados-videos-5"

# Crear directorio para guardar imágenes de depuración
debug_dir = os.path.join(ruta_carpeta_salida, "debug_images")
if not os.path.exists(debug_dir):
    os.makedirs(debug_dir)

# Función de preprocesamiento mejorada
def preprocess_image(img):
    # Convertir a escala de grises
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Aplicar un umbral adaptativo para binarizar
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    # Reducir ruido con un desenfoque suave
    blur = cv2.GaussianBlur(thresh, (5, 5), 0)
    # Operación morfológica para suavizar bordes gruesos
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.dilate(blur, kernel, iterations=1)
    morph = cv2.erode(morph, kernel, iterations=1)
    return morph

# Recorremos todos los archivos de video
for archivo in os.listdir(ruta_carpeta_videos):
    if archivo.endswith(".mp4"):  # o .avi, etc.
        nombre_base = os.path.splitext(archivo)[0]
        SOURCE_VIDEO_PATH = os.path.join(ruta_carpeta_videos, archivo)
        TARGET_VIDEO_PATH = os.path.join(ruta_carpeta_salida, f"{nombre_base}_procesado.mp4")
        nombre_excel = os.path.join(ruta_carpeta_salida, f"Conteo-total-{nombre_base}.xlsx")

        # Inicialización de contadores
        contador_selvalegre = 0
        contador_campeon = 0
        data = []  # Lista para rastrear IDs ya contados
        result_df = pd.DataFrame()

        # Instancias de tracking y anotación
        byte_tracker = BYTETracker(BYTETrackerArgs())
        video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
        generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
        box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=2)

        # Obtener primer frame para calcular dimensiones y línea
        frame = next(generator)
        w, h = frame.shape[1], frame.shape[0]
        linea_conteo = w // 2
        start_point = (linea_conteo, 0)
        end_point = (linea_conteo, h)

        # Reiniciar generador porque ya consumimos un frame
        generator = get_video_frames_generator(SOURCE_VIDEO_PATH)

        # Excel Workbook
        wb = Workbook()
        ws = wb.active
        ws.title = 'Contadores'

        # Procesamiento del video
        with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
            for frame in tqdm(generator, total=video_info.total_frames):
                results = model(frame)
                detections = Detections(
                    xyxy=results[0].boxes.xyxy.cpu().numpy(),
                    confidence=results[0].boxes.conf.cpu().numpy(),
                    class_id=results[0].boxes.cls.cpu().numpy().astype(int)
                )

                # Filtrado
                mask = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
                detections.filter(mask=mask, inplace=True)

                # Tracking
                tracks = byte_tracker.update(
                    output_results=detections2boxes(detections),
                    img_info=frame.shape,
                    img_size=frame.shape
                )

                tracker_id = match_detections_with_tracks(detections, tracks)
                detections.tracker_id = np.array(tracker_id)

                # Asegurar que sea tipo booleano
                mask = np.array([t is not None for t in detections.tracker_id], dtype=bool)
                detections.filter(mask=mask, inplace=True)

                # Etiquetas
                labels = [f" {CLASS_NAMES_DICT[class_id]}" for _, conf, class_id, tracker_id in detections]

                # Línea de conteo
                frame_with_line = cv2.line(frame, start_point, end_point, (255, 255, 255), 2)

                # Conteo personalizado
                for class_id, xyxy, tracker_id in zip(detections.class_id, detections.xyxy, detections.tracker_id):
                    x_center = int(xyxy[0] + (xyxy[2] - xyxy[0]) / 2)
                    if x_center >= linea_conteo:
                        if tracker_id not in data:
                            data.append(tracker_id)
                            # Extraemos la región con un margen
                            margin = 20  # Aumentamos el margen
                            crop_img = frame[max(0, int(xyxy[1])-margin):int(xyxy[3])+margin,
                                            max(0, int(xyxy[0])-margin):int(xyxy[2])+margin]

                            # Preprocesamiento
                            crop_img_processed = preprocess_image(crop_img)
                            # Redimensionar más para mejorar legibilidad
                            crop_img_resized = cv2.resize(crop_img_processed, None, fx=3, fy=3, interpolation=cv2.INTER_CUBIC)

                            # Guardar imagen para inspección
                            cv2.imwrite(os.path.join(debug_dir, f"debug_crop_{tracker_id}.jpg"), crop_img_resized)

                            # OCR con modo ajustado y diccionario de palabras
                            config = '--psm 11 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
                            ocr_text = pytesseract.image_to_string(crop_img_resized, config=config).strip().lower()
                            print(f"Tracker {tracker_id} - OCR: {ocr_text}")  # Para depuración

                            # Clasificación flexible
                            if "selvalegre" in ocr_text:
                                contador_selvalegre += 1
                            elif "campeon" in ocr_text or "campeón" in ocr_text:
                                contador_campeon += 1

                # Actualizar los contadores en el frame
                ancho_letra_blanca = 2
                ancho_letra_contorno = 4
                letra = 0.9

                cv2.putText(frame, f'Selvalegre: {contador_selvalegre}', (20, 30), cv2.FONT_HERSHEY_SIMPLEX, letra, (0, 0, 0), ancho_letra_contorno, cv2.LINE_AA)
                cv2.putText(frame, f'Selvalegre: {contador_selvalegre}', (20, 30), cv2.FONT_HERSHEY_SIMPLEX, letra, (255, 255, 255), ancho_letra_blanca)

                cv2.putText(frame, f'Campeon: {contador_campeon}', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, letra, (0, 0, 0), ancho_letra_contorno, cv2.LINE_AA)
                cv2.putText(frame, f'Campeon: {contador_campeon}', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, letra, (255, 255, 255), ancho_letra_blanca)

                frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
                box_annotator = BoxAnnotator(color=ColorPalette(), thickness=4, text_thickness=4, text_scale=1)
                sink.write_frame(frame)

        # Guardar en Excel
        result_df = pd.concat([result_df, pd.DataFrame({'Tipo': 'Selvalegre', 'Conteo Total': [contador_selvalegre]})], ignore_index=True)
        result_df = pd.concat([result_df, pd.DataFrame({'Tipo': 'Campeón', 'Conteo Total': [contador_campeon]})], ignore_index=True)

        for row in dataframe_to_rows(result_df, index=False, header=True):
            ws.append(row)

        chart = BarChart()
        chart.title = "Conteos Totales"
        chart.x_axis.title = 'Tipo'
        chart.y_axis.title = 'Contadores'
        data_ref = Reference(ws, min_col=2, min_row=1, max_row=result_df.shape[0]+1)
        labels = Reference(ws, min_col=1, min_row=2, max_row=result_df.shape[0]+1)
        chart.add_data(data_ref, titles_from_data=True)
        chart.set_categories(labels)
        ws.add_chart(chart, "D1")

        wb.save(nombre_excel)
        print(f"✅ Procesado y exportado: {archivo}")

  0%|          | 0/1177 [00:00<?, ?it/s]


0: 384x640 3 Selvalegres, 21.1ms
Speed: 4.2ms preprocess, 21.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Tracker 53 - OCR: aces

hj

al

aaa y

io

maga

an

es

ee

sas

aas

0: 384x640 3 Selvalegres, 22.1ms
Speed: 1.9ms preprocess, 22.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Selvalegres, 18.2ms
Speed: 2.0ms preprocess, 18.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Selvalegres, 17.5ms
Speed: 2.0ms preprocess, 17.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Selvalegres, 17.2ms
Speed: 2.0ms preprocess, 17.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Selvalegres, 16.2ms
Speed: 1.7ms preprocess, 16.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Selvalegres, 15.6ms
Speed: 1.6ms preprocess, 15.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4

KeyboardInterrupt: 